# KubeCon NA 2025 Demo - Environment Setup

This notebook sets up a complete Kubernetes environment with Crossplane and KubeVela for the demo.

## Prerequisites
- k3d installed
- kubectl installed
- helm installed
- Python 3.x with pip
- AWS credentials (optional, for AWS provider)

## Setup Steps
0. Check prerequisites and install Python packages (automated)
1. Load configuration
2. Create k3d cluster
3. Install Crossplane
4. Wait for Crossplane CRDs
5. Configure AWS Provider
6. Apply setup manifests
7. Install KubeVela

**Important:** Run the cells in order. The first cell will automatically install required Python packages from requirements.txt if they're not already installed.

**AWS Setup:** To use AWS resources, create a `.env.aws` file with your credentials before running Step 3.5.

**Note:** If you encounter errors, check that all prerequisites are installed and try re-running the failed cell.

In [ ]:
# Prerequisites Check and Setup
import sys
import subprocess
import os

print("=== Checking Prerequisites ===\n")

# Check Python packages
print("1. Checking Python packages...")
try:
    import yaml
    print("   ✓ PyYAML is installed")
except ImportError:
    print("   ✗ PyYAML is NOT installed")
    print("   Installing required packages from requirements.txt...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-r", "requirements.txt"])
    print("   ✓ Packages installed successfully")
    import yaml

# Check if config.yaml exists
print("\n2. Checking configuration file...")
if os.path.exists('config.yaml'):
    print("   ✓ config.yaml found")
else:
    print("   ✗ config.yaml NOT found")
    raise FileNotFoundError("config.yaml is missing. Please ensure it exists in the current directory.")

# Check command-line tools
print("\n3. Checking required tools...")
tools = {
    'k3d': 'k3d version',
    'kubectl': 'kubectl version --client --short',
    'helm': 'helm version --short',
    'vela': 'vela version'
}

all_tools_ok = True
for tool, cmd in tools.items():
    try:
        result = subprocess.run(cmd.split(), capture_output=True, text=True, timeout=5)
        if result.returncode == 0:
            print(f"   ✓ {tool} is installed")
        else:
            print(f"   ✗ {tool} is NOT working properly")
            all_tools_ok = False
    except (subprocess.TimeoutExpired, FileNotFoundError):
        print(f"   ✗ {tool} is NOT installed")
        all_tools_ok = False

if not all_tools_ok:
    print("\n⚠️  WARNING: Some tools are missing. Please install them before proceeding.")
    print("   - k3d: https://k3d.io/")
    print("   - kubectl: https://kubernetes.io/docs/tasks/tools/")
    print("   - helm: https://helm.sh/docs/intro/install/")
    print("   - vela: https://kubevela.io/docs/installation/kubernetes/#install-vela-cli")
else:
    print("\n✓ All prerequisites are satisfied!")
    print("Ready to proceed with the setup.")

In [ ]:
import yaml
import os

# Load configuration from config.yaml
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Extract values for use in notebook
cluster_name = config['cluster']['name']
api_port = config['cluster']['api_port']
http_port = config['cluster']['http_port']
crossplane_namespace = config['crossplane']['namespace']
min_crds = config['crossplane']['min_crds']
setup_dir = config['setup']['manifests_dir']

# Store in environment for bash cells
os.environ['CLUSTER_NAME'] = cluster_name
os.environ['API_PORT'] = str(api_port)
os.environ['HTTP_PORT'] = str(http_port)
os.environ['CROSSPLANE_NAMESPACE'] = crossplane_namespace
os.environ['MIN_CRDS'] = str(min_crds)
os.environ['SETUP_DIR'] = setup_dir

# Also write to a shell file that can be sourced
with open('.env.sh', 'w') as f:
    f.write(f'export CLUSTER_NAME="{cluster_name}"\n')
    f.write(f'export API_PORT="{api_port}"\n')
    f.write(f'export HTTP_PORT="{http_port}"\n')
    f.write(f'export CROSSPLANE_NAMESPACE="{crossplane_namespace}"\n')
    f.write(f'export MIN_CRDS="{min_crds}"\n')
    f.write(f'export SETUP_DIR="{setup_dir}"\n')

print("Configuration loaded successfully:")
print(f"  Cluster name: {cluster_name}")
print(f"  API port: {api_port}")
print(f"  HTTP port: {http_port}")
print(f"  Crossplane namespace: {crossplane_namespace}")
print(f"  Minimum CRDs: {min_crds}")
print(f"  Setup directory: {setup_dir}")
print("\nEnvironment variables set and saved to .env.sh")
print("Ready to proceed with setup!")

## Step 1: Create k3d Cluster

Creating a lightweight Kubernetes cluster using k3d with custom port mappings.

In [ ]:
%%bash
set -e  # Exit on error
source .env.sh  # Load configuration

echo "=== Step 1: Creating k3d cluster with local registry ==="

# Delete existing cluster if it exists
echo "Cleaning up any existing cluster..."
k3d cluster delete $CLUSTER_NAME 2>/dev/null || echo "No existing cluster to delete"

# Delete existing registry if it exists
echo "Cleaning up any existing registry..."
k3d registry delete registry.localhost 2>/dev/null || echo "No existing registry to delete"

# Create registry first
echo ""
echo "Creating local Docker registry..."
if k3d registry create registry.localhost --port 0.0.0.0:5000; then
    echo "✓ Registry created successfully at localhost:5000"
else
    echo "✗ Failed to create registry"
    exit 1
fi

# Create cluster and connect it to the registry
echo ""
echo "Creating k3d cluster: $CLUSTER_NAME"
if k3d cluster create $CLUSTER_NAME \
    --api-port $API_PORT \
    -p "${HTTP_PORT}:80@loadbalancer" \
    --k3s-arg="--kubelet-arg=max-open-files=1000000@server:*" \
    --registry-use k3d-registry.localhost:5000 \
    --wait; then
    echo "✓ Cluster created successfully"
else
    echo "✗ Failed to create cluster"
    exit 1
fi

# IMPORTANT: Set kubectl context to the new cluster
echo ""
echo "Setting kubectl context to k3d-$CLUSTER_NAME..."
kubectl config use-context "k3d-$CLUSTER_NAME"

# Verify cluster is accessible
echo "Verifying cluster access..."
if kubectl cluster-info &>/dev/null; then
    echo "✓ Cluster is accessible"
    echo "Current context: $(kubectl config current-context)"
    kubectl get nodes
else
    echo "✗ Cannot access cluster"
    exit 1
fi

# Verify registry
echo ""
echo "=== Registry Setup Complete ==="
echo "Registry URL: localhost:5000"
echo "Registry status:"
k3d registry list
echo ""
docker ps | grep registry || echo "Warning: Registry container not visible"
echo ""
echo "To push images: docker tag <image> localhost:5000/<image>:<tag>"
echo "                docker push localhost:5000/<image>:<tag>"
echo ""
echo "In k3d cluster, use: k3d-registry.localhost:5000/<image>:<tag>"

## Step 2: Install Crossplane

Installing Crossplane for infrastructure orchestration and composition.

In [ ]:
%%bash
set -e  # Exit on error
source .env.sh  # Load configuration

echo "=== Step 2: Installing Crossplane ==="

# Add and update helm repo
echo "Adding Crossplane helm repository..."
helm repo add crossplane-stable https://charts.crossplane.io/stable 2>/dev/null || echo "Repository already exists"
helm repo update

# Check if Crossplane is already installed
if helm list -n $CROSSPLANE_NAMESPACE | grep -q crossplane; then
    echo "⚠ Crossplane is already installed. Upgrading..."
    HELM_CMD="upgrade"
else
    echo "Installing Crossplane..."
    HELM_CMD="install"
fi

# Install or upgrade Crossplane
if helm $HELM_CMD crossplane crossplane-stable/crossplane \
    --namespace $CROSSPLANE_NAMESPACE \
    --create-namespace \
    --wait \
    --timeout 10m; then
    echo "✓ Crossplane helm chart $HELM_CMD completed"
else
    echo "✗ Failed to $HELM_CMD Crossplane"
    exit 1
fi

# Wait for pods to be ready
echo "Waiting for Crossplane pods to be ready..."
if kubectl wait --namespace $CROSSPLANE_NAMESPACE \
    --for=condition=ready pod \
    --selector=app.kubernetes.io/component=cloud-infrastructure-controller \
    --timeout=1200s; then
    echo "✓ Crossplane controller is ready"
else
    echo "✗ Crossplane controller failed to become ready"
    exit 1
fi

echo "Crossplane installation complete!"
kubectl get pods -n $CROSSPLANE_NAMESPACE

## Step 3: Wait for Crossplane CRDs

Crossplane installs various Custom Resource Definitions (CRDs) that are needed for the next steps. This cell waits until the minimum number of CRDs are available.

In [ ]:
%%bash
set -e  # Exit on error
source .env.sh  # Load configuration

echo "=== Step 3: Waiting for Crossplane CRDs ==="

MAX_RETRIES=60
RETRY_DELAY=5

echo "Waiting for at least $MIN_CRDS Crossplane CRDs to be installed..."

for i in $(seq 1 $MAX_RETRIES); do
    CRD_COUNT=$(kubectl api-resources | grep crossplane | wc -l)
    echo "Attempt $i/$MAX_RETRIES: Found $CRD_COUNT Crossplane CRDs"
    
    if [ $CRD_COUNT -ge $MIN_CRDS ]; then
        echo "✓ Sufficient CRDs are available ($CRD_COUNT >= $MIN_CRDS)"
        break
    fi
    
    if [ $i -eq $MAX_RETRIES ]; then
        echo "✗ Timeout: Only $CRD_COUNT CRDs found after ${MAX_RETRIES} attempts"
        exit 1
    fi
    
    sleep $RETRY_DELAY
done

echo ""
echo "Current Crossplane pods:"
kubectl get pods -n $CROSSPLANE_NAMESPACE

echo ""
echo "Sample Crossplane CRDs:"
kubectl api-resources | grep crossplane | head -10

In [ ]:
%%bash
set -e  # Exit on error
source .env.sh  # Load configuration

echo "=== Step 3.5: Configuring AWS Provider ==="

# Check if .env.aws file exists
if [ ! -f "../.env.aws" ]; then
    echo "⚠ Warning: .env.aws file not found"
    echo "Creating template .env.aws file..."
    cat > ../.env.aws << 'EOF'
# AWS Credentials for Crossplane
AWS_ACCESS_KEY_ID=your-access-key-id
AWS_SECRET_ACCESS_KEY=your-secret-access-key
AWS_DEFAULT_REGION=us-west-2
EOF
    echo "✓ Template created. Please edit .env.aws with your credentials and re-run this cell."
    exit 0
fi

# Source AWS credentials
source ../.env.aws

# Check if credentials are set
if [ "$AWS_ACCESS_KEY_ID" == "your-access-key-id" ] || [ -z "$AWS_ACCESS_KEY_ID" ]; then
    echo "⚠ Warning: AWS credentials not configured in .env.aws"
    echo "Please edit .env.aws with your actual AWS credentials and re-run this cell."
    exit 0
fi

echo "AWS credentials found, configuring Crossplane..."

# Install AWS Provider
echo "1. Installing Crossplane AWS Provider..."
cat <<EOF | kubectl apply -f -
apiVersion: pkg.crossplane.io/v1
kind: Provider
metadata:
  name: upbound-provider-aws-dynamodb
spec:
  package: xpkg.upbound.io/upbound/provider-aws-dynamodb:v1.23.2
---
apiVersion: pkg.crossplane.io/v1
kind: Provider
metadata:
  name: upbound-provider-aws-s3
spec:
  package: xpkg.upbound.io/upbound/provider-aws-s3:v1.23.2
EOF

echo "   Waiting for provider to be installed..."
kubectl wait --for=condition=installed --timeout=300s provider.pkg.crossplane.io/upbound-provider-aws-dynamodb
kubectl wait --for=condition=installed --timeout=300s provider.pkg.crossplane.io/upbound-provider-aws-s3

echo "   Waiting for provider to be healthy..."
kubectl wait --for=condition=healthy --timeout=300s provider.pkg.crossplane.io/upbound-provider-aws-dynamodb
kubectl wait --for=condition=healthy --timeout=300s provider.pkg.crossplane.io/upbound-provider-aws-s3

echo "✓ AWS Provider installed"

# Create Kubernetes secret with AWS credentials
echo ""
echo "2. Creating Kubernetes secret with AWS credentials..."
# Create credentials string with session token if available
if [ -n "$AWS_SESSION_TOKEN" ]; then
    CREDENTIALS_STRING="[default]
aws_access_key_id = ${AWS_ACCESS_KEY_ID}
aws_secret_access_key = ${AWS_SECRET_ACCESS_KEY}
aws_session_token = ${AWS_SESSION_TOKEN}"
    echo "   Including session token for temporary credentials"
else
    CREDENTIALS_STRING="[default]
aws_access_key_id = ${AWS_ACCESS_KEY_ID}
aws_secret_access_key = ${AWS_SECRET_ACCESS_KEY}"
    echo "   Using long-term credentials (no session token)"
fi

kubectl create secret generic aws-credentials \
    -n $CROSSPLANE_NAMESPACE \
    --from-literal=credentials="$CREDENTIALS_STRING" \
    --dry-run=client -o yaml | kubectl apply -f -

echo "✓ AWS credentials secret created"

# Create ProviderConfig
echo ""
echo "3. Creating ProviderConfig for AWS..."
cat <<EOF | kubectl apply -f -
apiVersion: aws.upbound.io/v1beta1
kind: ProviderConfig
metadata:
  name: default
spec:
  credentials:
    source: Secret
    secretRef:
      namespace: $CROSSPLANE_NAMESPACE
      name: aws-credentials
      key: credentials
EOF

echo "✓ ProviderConfig created"

echo ""
echo "=== AWS Provider Configuration Complete ==="
echo "✓ Provider: provider-aws-dynamodb"
echo "✓ Credentials: Configured from .env.aws"
echo "✓ Region: ${AWS_DEFAULT_REGION}"

## Step 3.5: Configure AWS Provider for Crossplane

Configure Crossplane to use AWS credentials from `.env.aws` file.

**Important:** Make sure you have filled in your AWS credentials in the `.env.aws` file before running this cell.

## Step 4: Apply Setup Manifests

Applying Crossplane providers, configurations, and compositions from the setup directory.

In [ ]:
%%bash
set -e  # Exit on error
source .env.sh  # Load configuration

echo "=== Step 4: Applying Setup Manifests ==="

# Check if setup directory exists
if [ ! -d "$SETUP_DIR" ]; then
    echo "⚠ Warning: Setup directory '$SETUP_DIR' not found"
    echo "Creating placeholder directory..."
    mkdir -p "$SETUP_DIR"
    echo "Please add your Crossplane provider and configuration files to the '$SETUP_DIR' directory"
    echo "Then re-run this cell to apply them"
    exit 0
fi

# Check if directory has any yaml files
if [ -z "$(find $SETUP_DIR -maxdepth 1 -name "*.yaml" -o -name "*.yml" 2>/dev/null)" ]; then
    echo "⚠ Warning: No YAML files found in '$SETUP_DIR' directory"
    echo "Skipping manifest application"
    exit 0
fi

# Apply manifests
echo "Applying manifests from $SETUP_DIR..."
if kubectl apply -f $SETUP_DIR/; then
    echo "✓ Initial manifests applied"
else
    echo "⚠ Some manifests may have failed to apply (CRDs might not be ready yet)"
fi

# Wait for provider configs CRD to be available
echo ""
echo "Waiting for providerconfigs CRD to be available..."
MAX_RETRIES=60
for i in $(seq 1 $MAX_RETRIES); do
    if kubectl api-resources | grep crossplane | grep -q providerconfigs; then
        echo "✓ ProviderConfigs CRD is available"
        break
    fi
    
    if [ $i -eq $MAX_RETRIES ]; then
        echo "⚠ Warning: providerconfigs CRD not found, but continuing..."
        exit 0
    fi
    
    sleep 5
done

# Wait for function pods
echo ""
echo "Waiting for Crossplane function pods..."
if kubectl wait --namespace $CROSSPLANE_NAMESPACE \
    --for=condition=ready pod \
    --selector=pkg.crossplane.io/function=function-patch-and-transform \
    --timeout=1200s 2>/dev/null; then
    echo "✓ Function pods are ready"
else
    echo "⚠ Function pods not found or not ready yet (may not be installed)"
fi

# Wait for provider pods
echo ""
echo "Waiting for Crossplane provider pods..."
if kubectl wait --namespace $CROSSPLANE_NAMESPACE \
    --for=condition=ready pod \
    --selector=pkg.crossplane.io/provider=provider-kubernetes \
    --timeout=1200s 2>/dev/null; then
    echo "✓ Provider pods are ready"
else
    echo "⚠ Provider pods not found or not ready yet (may not be installed)"
fi

# Re-apply manifests to ensure everything is configured
echo ""
echo "Re-applying manifests to ensure configuration..."
kubectl apply -f $SETUP_DIR/ 2>/dev/null || echo "⚠ Some resources may already exist"

echo ""
echo "✓ Crossplane setup complete!"

## Step 5: Install KubeVela

Installing KubeVela for application delivery and management.

In [ ]:
%%bash
set -e  # Exit on error
source .env.sh  # Load configuration

echo "=== Step 5: Installing KubeVela ==="

# Add KubeVela helm repository
echo "Adding KubeVela helm repository..."
helm repo add kubevela https://charts.kubevela.net/core 2>/dev/null || echo "Repository already exists"
helm repo update

# Check if KubeVela is already installed
if helm list -n vela-system | grep -q kubevela; then
    echo "⚠ KubeVela is already installed. Upgrading..."
    HELM_CMD="upgrade"
else
    echo "Installing KubeVela..."
    HELM_CMD="install"
fi

# Install or upgrade KubeVela
if helm $HELM_CMD kubevela kubevela/vela-core \
    --create-namespace \
    -n vela-system \
    --wait \
    --timeout 10m; then
    echo "✓ KubeVela helm chart $HELM_CMD completed"
else
    echo "✗ Failed to $HELM_CMD KubeVela"
    exit 1
fi

# Wait for KubeVela pods to be ready
echo "Waiting for KubeVela pods to be ready..."
if kubectl wait --namespace vela-system \
    --for=condition=ready pod \
    --selector=app.kubernetes.io/name=vela-core \
    --timeout=600s; then
    echo "✓ KubeVela controller is ready"
else
    echo "✗ KubeVela controller failed to become ready"
    exit 1
fi

echo ""
echo "KubeVela installation complete!"
kubectl get pods -n vela-system

echo ""
echo "Checking KubeVela version..."
kubectl get deployment -n vela-system kubevela-vela-core -o jsonpath='{.spec.template.spec.containers[0].image}'
echo ""

echo ""
echo "Installing velaux..."
vela addon enable velaux
echo ""

# port forward velaux
vela port-forward -n vela-system addon-velaux 8000:8000 &
# open http://localhost:8000

Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling connection for 8000
Handling conne

## Setup Complete!

Your KubeCon demo environment is now ready. The following components have been installed:

- **k3d cluster**: A lightweight Kubernetes cluster for local development
- **Crossplane**: Infrastructure orchestration and composition framework
- **KubeVela**: Application delivery and management platform
- **Custom configurations**: Any providers and compositions from the setup directory

### Next Steps

1. Explore other notebooks in this directory for demo scenarios
2. Check cluster status: `kubectl get pods -A`
3. View Crossplane resources: `kubectl get crossplane`
4. View KubeVela applications: `kubectl get applications -A`
5. When finished, run the cleanup notebook: `00-Env-cleanup.ipynb`

### Troubleshooting

If you encountered errors:
- Ensure all prerequisites are installed (k3d, kubectl, helm)
- Check that ports 6443 and 8090 are available
- Review pod logs:
  - Crossplane: `kubectl logs -n crossplane-system <pod-name>`
  - KubeVela: `kubectl logs -n vela-system <pod-name>`
- Try re-running failed cells after investigating the issue

For cleanup and teardown, use the `00-Env-cleanup.ipynb` notebook.